# In this notebook I will create new features

#### Import necessary libraries

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime, timedelta
import ast
from itertools import combinations

#### Read cleaned and filled data

In [20]:
data = pd.read_csv('data_filled.csv')

In [21]:
data.head(2)

,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,Paris,"['European', 'French', 'International']",5570.0,3.5,medium,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,Stockholm,no data,1537.0,4.0,no_data,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


## Action plan:

1) From the Reviews column make a) the difference in days between the dates and b) the polarity of the review

2) Make the branches column from the URL_TA and ID_TA columns

3) Create a Cuisine Style 'length' Column

4) Get dummy variables

5) ...

# Step №1

In [22]:
# make list of lists
data.loc[data['Reviews'].notna(), 'Reviews'] = data.loc[data['Reviews'].notna(),
                                                        'Reviews'].apply(lambda x: ast.literal_eval(x))

In [23]:
def date_finder(val):
    '''It takes a value and returns a date string from it using regular expression. In case it is nan returns nan'''
    if np.any(pd.notna(val)):
        val = str(val)
        return re.findall(r'(\d+/\d+/\d+)', val)
    else:
        return np.nan

In [24]:
data['days_diff'] = data['Reviews'].apply(date_finder)

In [25]:
data['days_diff'].apply(lambda x: len(x) if np.any(
    pd.notna(x)) else np.nan).value_counts()
# there are values with two, one and three dates

2.0    28969
1.0     4556
3.0        2
Name: days_diff, dtype: int64

Due to the fact that there are values with 1,2 and even 3 dates, it will not be possible to create the 'difference between dates in days' feature. But I can find the earliest date and create a feature 'days from the earliest date in the dataset to the earliest date in the list of dates'

I will skip part with datws for now and do polarity

In [26]:
data['Rating'].value_counts()

4.0    13692
4.5    10929
3.5     6824
5.0     3879
3.0     2955
2.5      915
2.0      466
1.0      201
1.5      139
Name: Rating, dtype: int64

In [27]:
def polarity_maker(val):
    if 0 < val <= 2:
        return 'negative'
    elif 2 < val <= 3.5:
        return 'neutral'
    elif 3.5 < val <= 5:
        return 'positive'

In [28]:
data['polarity'] = data['Rating'].apply(polarity_maker)  # successul

In [29]:
del data['Reviews']
del data['days_diff']
data.head(2)

,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,URL_TA,ID_TA,polarity
0,Paris,"['European', 'French', 'International']",5570.0,3.5,medium,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,neutral
1,Stockholm,no data,1537.0,4.0,no_data,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,positive


# Step №2

URL_TA & ID_TA Duplicates in these columns are the same - refer to the same observations, respectively

In [30]:
# list of IDs wich have branches
branches_ids = data['ID_TA'].value_counts()[:20].index.tolist()


def branches(cell):
    '''It return 1 if there are >1 restauraunts with this ID, otherwise returns 0'''
    if cell in branches_ids:
        return 1
    else:
        return 0

In [31]:
data['branches'] = data['ID_TA'].apply(branches)

In [32]:
del data['URL_TA']
del data['ID_TA']
data.head(2)  # good

,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,polarity,branches
0,Paris,"['European', 'French', 'International']",5570.0,3.5,medium,194.0,neutral,0
1,Stockholm,no data,1537.0,4.0,no_data,10.0,positive,0


# Step №3

In [33]:
# make list of lists
data.loc[data['Cuisine Style'] != 'no data', 'Cuisine Style'] = data.loc[data['Cuisine Style'] != 'no data',
                                                                         'Cuisine Style'].apply(lambda x: ast.literal_eval(x))

In [34]:
def len_finder(val):
    '''It returns number of cuisine styles in restaurant or 1 in case isna==True'''
    if val != 'no data':
        return len(val)
    else:
        return 1

In [35]:
data['n_styles'] = data['Cuisine Style'].apply(len_finder)

In [36]:
del data['Cuisine Style']
data.head(2)  # good

,City,Ranking,Rating,Price Range,Number of Reviews,polarity,branches,n_styles
0,Paris,5570.0,3.5,medium,194.0,neutral,0,3
1,Stockholm,1537.0,4.0,no_data,10.0,positive,0,1


# Step №4

Get dummy variables

In [37]:
data.head()

,City,Ranking,Rating,Price Range,Number of Reviews,polarity,branches,n_styles
0,Paris,5570.0,3.5,medium,194.0,neutral,0,3
1,Stockholm,1537.0,4.0,no_data,10.0,positive,0,1
2,London,353.0,4.5,high,688.0,positive,0,7
3,Berlin,3458.0,5.0,no_data,3.0,positive,0,1
4,Munich,621.0,4.0,medium,84.0,positive,0,3


In [38]:
df_model = pd.get_dummies(
    data, columns=['polarity', 'Price Range', 'City'], prefix='', prefix_sep='')

In [39]:
df_model.head(2)

,Ranking,Rating,Number of Reviews,branches,n_styles,negative,neutral,positive,high,low,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570.0,3.5,194.0,0,3,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,10.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


# Polynomial features

In [40]:
df_model.head(2)

,Ranking,Rating,Number of Reviews,branches,n_styles,negative,neutral,positive,high,low,...,Munich,Oporto,Oslo,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich
0,5570.0,3.5,194.0,0,3,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,1537.0,4.0,10.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [41]:
df_model['Ranking_squared'] = df_model['Ranking']**2
df_model['NoR_squared'] = df_model['Number of Reviews']**2
df_model['Rank_NoR_multi'] = df_model['Ranking'] * \
    df_model['Number of Reviews']

In [42]:
df_model.head()

,Ranking,Rating,Number of Reviews,branches,n_styles,negative,neutral,positive,high,low,...,Paris,Prague,Rome,Stockholm,Vienna,Warsaw,Zurich,Ranking_squared,NoR_squared,Rank_NoR_multi
0,5570.0,3.5,194.0,0,3,0,1,0,0,0,...,1,0,0,0,0,0,0,31024900.0,37636.0,1080580.0
1,1537.0,4.0,10.0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,2362369.0,100.0,15370.0
2,353.0,4.5,688.0,0,7,0,0,1,1,0,...,0,0,0,0,0,0,0,124609.0,473344.0,242864.0
3,3458.0,5.0,3.0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,11957764.0,9.0,10374.0
4,621.0,4.0,84.0,0,3,0,0,1,0,0,...,0,0,0,0,0,0,0,385641.0,7056.0,52164.0


# Summary

- Deleted all non numerical features

- Added extra fetureas like: branches, n_styles, polarity

- Added polynomial features

- Got dummies for polarity, Price Range and City features

- total shape: (5,46)

In [43]:
df_model.to_csv('data_model.csv', index=False)  # save model data